In [11]:
import requests
from bs4 import BeautifulSoup
import re
import json
import html
import pandas as pd
import datetime
import time
import random

from tqdm import tqdm

pd.set_option('display.max_columns', None) 

In [12]:
df = pd.read_csv('xe_20230601.csv')

In [25]:
df.head(5)

,id,internal_id,item_type,transaction_type,address,geo_lat,geo_lng,ad_group_id,levels,bedrooms,bathrooms,construction_year,display_golden_border,owner_logo,price_per_unit_area,account_id,company_title,is_commercial,sponsored,title,title_abbreviation,price,price_with_abbreviation,price_per_square_meter,size_with_square_meter,date,extra_seo_info,image_alt,image_gallery,url,unique_group_url,extraction_date
0,815053331,47143151,re_residence,LET.NORMAL,Αθήνα (Λυκαβηττός),37.979777,23.742884,NaN,"['Υπόγειο', 'Ισόγειο']",1.0,1.0,1970.0,True,NaN,10,2267566.0,NaN,False,False,Διαμέρισμα 54 τ.μ.,Διαμέρισμα 54 τ.μ.,560 €,560 €,10 €/τ.μ.,54 τ.μ.,πριν από 2 ημέρες,Ενοικίαση κατοικίας,Ενοικίαση κατοικίας Αθήνα (Λυκαβηττός) Διαμέρι...,[{'medium': {'webp': 'https://a2.xe.gr/2023053...,https://www.xe.gr/property/d/enoikiaseis-katoi...,NaN,2023-06-01
1,815047933,47142491,re_residence,LET.NORMAL,Αθήνα (Άνω Πατήσια),38.017481,23.735684,459093.0,['1ος'],1.0,1.0,1969.0,True,NaN,9,1407351.0,NaN,False,False,Διαμέρισμα 45 τ.μ.,Διαμέρισμα 45 τ.μ.,420 €,420 €,9 €/τ.μ.,45 τ.μ.,πριν από 2 ημέρες,Ενοικίαση κατοικίας,Ενοικίαση κατοικίας Αθήνα (Άνω Πατήσια) Διαμέρ...,[{'medium': {'webp': 'https://a2.xe.gr/2023053...,https://www.xe.gr/property/d/enoikiaseis-katoi...,https://www.xe.gr/property/u/459093,2023-06-01
2,815017929,47140206,re_residence,LET.NORMAL,Αθήνα (Κουντουριώτικα),37.992291,23.755579,NaN,['2ος'],1.0,1.0,1975.0,True,NaN,13,110841.0,NaN,True,False,Διαμέρισμα 60 τ.μ.,Διαμέρισμα 60 τ.μ.,780 €,780 €,13 €/τ.μ.,60 τ.μ.,πριν από 2 ημέρες,Ενοικίαση κατοικίας,Ενοικίαση κατοικίας Αθήνα (Κουντουριώτικα) Δια...,[{'medium': {'webp': 'https://a2.xe.gr/2023053...,https://www.xe.gr/property/d/enoikiaseis-katoi...,NaN,2023-06-01
3,814952950,47138240,re_residence,LET.NORMAL,Αθήνα (Πολύγωνο),37.999083,23.755417,NaN,['1ος'],1.0,1.0,1981.0,True,NaN,11,1344554.0,NaN,False,False,Διαμέρισμα 55 τ.μ.,Διαμέρισμα 55 τ.μ.,600 €,600 €,11 €/τ.μ.,55 τ.μ.,πριν από 3 ημέρες,Ενοικίαση κατοικίας,Ενοικίαση κατοικίας Αθήνα (Πολύγωνο) Διαμέρισμ...,[{'medium': {'webp': 'https://a2.xe.gr/2023053...,https://www.xe.gr/property/d/enoikiaseis-katoi...,NaN,2023-06-01
4,814956607,47138448,re_residence,LET.NORMAL,Αθήνα (Πανόρμου),37.990937,23.765072,NaN,['1ος'],1.0,1.0,1970.0,True,https://a2.xe.gr/LOGO/1620977238222.jpg,16,1220409.0,BLUEGROUND REAL ESTATE,True,False,Διαμέρισμα 50 τ.μ.,Διαμέρισμα 50 τ.μ.,790 €,790 €,16 €/τ.μ.,50 τ.μ.,πριν από 3 ημέρες,Ενοικίαση κατοικίας,Ενοικίαση κατοικίας Αθήνα (Πανόρμου) Διαμέρισμ...,[{'medium': {'webp': 'https://a2.xe.gr/2023053...,https://www.xe.gr/property/d/enoikiaseis-katoi...,NaN,2023-06-01


In [14]:
df.shape

(5842, 32)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5842 entries, 0 to 5841
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       5842 non-null   int64  
 1   internal_id              5842 non-null   int64  
 2   item_type                5842 non-null   object 
 3   transaction_type         5842 non-null   object 
 4   address                  5842 non-null   object 
 5   geo_lat                  5842 non-null   float64
 6   geo_lng                  5842 non-null   float64
 7   ad_group_id              762 non-null    float64
 8   levels                   5842 non-null   object 
 9   bedrooms                 5545 non-null   float64
 10  bathrooms                5229 non-null   float64
 11  construction_year        4743 non-null   float64
 12  display_golden_border    5842 non-null   bool   
 13  owner_logo               4493 non-null   object 
 14  price_per_unit_area     

In [16]:
df.describe(include = 'object')

,item_type,transaction_type,address,levels,owner_logo,company_title,title,title_abbreviation,price,price_with_abbreviation,price_per_square_meter,size_with_square_meter,date,extra_seo_info,image_alt,image_gallery,url,unique_group_url,extraction_date
count,5842,5842,5842,5842,4493,4494,5842,5842,5842,5842,5841,5842,2740,5842,5842,5842,5842,762,5842
unique,1,1,238,59,570,568,422,422,264,251,65,286,39,1,4863,5128,5825,761,1
top,re_residence,LET.NORMAL,Αθήνα (Κολωνάκι),['1ος'],https://a2.xe.gr/LOGO/1622120851959.jpg,Golden Home,Διαμέρισμα 50 τ.μ.,Διαμέρισμα 50 τ.μ.,550 €,550 €,8 €/τ.μ.,50 τ.μ.,πριν από 2 εβδομάδες,Ενοικίαση κατοικίας,Ενοικίαση κατοικίας Αθήνα (Κολωνάκι) Διαμέρισμ...,[],https://www.xe.gr/property/d/enoikiaseis-katoi...,https://www.xe.gr/property/u/452971,2023-06-01
freq,5842,5842,241,1249,314,314,290,290,311,313,817,292,598,5842,7,701,2,2,5842


In [20]:
df.describe()

,id,internal_id,geo_lat,geo_lng,ad_group_id,bedrooms,bathrooms,construction_year,price_per_unit_area,account_id
count,5.842000e+03,5.842000e+03,5842.000000,5842.000000,762.000000,5545.000000,5229.000000,4743.000000,5842.000000,5.741000e+03
mean,7.960085e+08,4.643847e+07,37.986546,23.739937,427542.146982,1.864743,1.195831,1976.257854,11.859124,1.205475e+06
std,6.715492e+07,2.008976e+06,0.019499,0.020774,51775.922710,1.243349,0.738009,16.094772,52.817211,7.211518e+05
min,5.765660e+05,2.009286e+07,37.948830,23.687081,55188.000000,1.000000,1.000000,1900.000000,0.000000,2.003400e+04
25%,8.045269e+08,4.657072e+07,37.971772,23.727186,427821.500000,1.000000,1.000000,1970.000000,8.000000,5.310090e+05
50%,8.123143e+08,4.698676e+07,37.986437,23.739657,444977.500000,2.000000,1.000000,1975.000000,9.000000,1.283475e+06
75%,8.142003e+08,4.710355e+07,38.000439,23.753961,453970.500000,2.000000,1.000000,1980.000000,12.000000,1.879279e+06
max,8.152422e+08,4.715202e+07,38.032695,23.789568,459980.000000,21.000000,16.000000,2023.000000,3205.000000,2.268029e+06


In [26]:
useless_columns = ['item_type', 'transaction_type', 'owner_logo', 'title', 
                   'price_with_abbreviation', 'extra_seo_info', 'image_alt', 
                   'image_gallery', 'url', 'unique_group_url', 'price_per_square_meter', 'display_golden_border',
                   'is_commercial', 'sponsored']

In [29]:
useful_columns = ['id', 'internal_id', 'title_abbreviation','address','price','price_per_unit_area',
                  'size_with_square_meter','construction_year','levels','bedrooms', 'bathrooms',  
                   'date', 'extraction_date', 'geo_lat', 'geo_lng','company_title', 'account_id','ad_group_id']

In [30]:
len(useless_columns)+len(useful_columns)

32

In [32]:
df_useful = df.loc[:, useful_columns]

In [33]:
df_useful

,id,internal_id,title_abbreviation,address,price,price_per_unit_area,size_with_square_meter,construction_year,levels,bedrooms,bathrooms,date,extraction_date,geo_lat,geo_lng,company_title,account_id,ad_group_id
0,815053331,47143151,Διαμέρισμα 54 τ.μ.,Αθήνα (Λυκαβηττός),560 €,10,54 τ.μ.,1970.0,"['Υπόγειο', 'Ισόγειο']",1.0,1.0,πριν από 2 ημέρες,2023-06-01,37.979777,23.742884,NaN,2267566.0,NaN
1,815047933,47142491,Διαμέρισμα 45 τ.μ.,Αθήνα (Άνω Πατήσια),420 €,9,45 τ.μ.,1969.0,['1ος'],1.0,1.0,πριν από 2 ημέρες,2023-06-01,38.017481,23.735684,NaN,1407351.0,459093.0
2,815017929,47140206,Διαμέρισμα 60 τ.μ.,Αθήνα (Κουντουριώτικα),780 €,13,60 τ.μ.,1975.0,['2ος'],1.0,1.0,πριν από 2 ημέρες,2023-06-01,37.992291,23.755579,NaN,110841.0,NaN
3,814952950,47138240,Διαμέρισμα 55 τ.μ.,Αθήνα (Πολύγωνο),600 €,11,55 τ.μ.,1981.0,['1ος'],1.0,1.0,πριν από 3 ημέρες,2023-06-01,37.999083,23.755417,NaN,1344554.0,NaN
4,814956607,47138448,Διαμέρισμα 50 τ.μ.,Αθήνα (Πανόρμου),790 €,16,50 τ.μ.,1970.0,['1ος'],1.0,1.0,πριν από 3 ημέρες,2023-06-01,37.990937,23.765072,BLUEGROUND REAL ESTATE,1220409.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5837,798078305,46276831,Διαμέρισμα 150 τ.μ.,Βύρωνας,1.050 €,7,150 τ.μ.,1974.0,['3ος'],3.0,1.0,NaN,2023-06-01,37.955561,23.750699,Homenet Real Estate & Development,1161699.0,NaN
5838,799256815,46334139,Διαμέρισμα 150 τ.μ.,Ζωγράφου (Κέντρο),1.000 €,7,150 τ.μ.,NaN,['3ος'],3.0,1.0,NaN,2023-06-01,37.976858,23.770422,Dimit Estate,1144164.0,NaN
5839,804971100,46592632,Μονοκατοικία 600 τ.μ.,Φιλοθέη,8.000 €,13,600 τ.μ.,NaN,['Ισόγειο'],4.0,3.0,NaN,2023-06-01,38.023565,23.777302,Moisiadis Real Estate,451111.0,NaN
5840,2550572,21407274,Διαμέρισμα 50 τ.μ.,Βύρωνας (Αγορά),590 €,12,50 τ.μ.,1970.0,['1ος'],2.0,NaN,NaN,2023-06-01,37.959260,23.756369,NaN,88626.0,NaN


In [37]:
df_useful['address'].unique()

array(['Αθήνα (Λυκαβηττός)', 'Αθήνα (Άνω Πατήσια)',
       'Αθήνα (Κουντουριώτικα)', 'Αθήνα (Πολύγωνο)', 'Αθήνα (Πανόρμου)',
       'Αθήνα (Λόφος Σκουζέ)', 'Αθήνα (Κάτω Πατήσια)', 'Αθήνα (Μετς)',
       'Αθήνα (Ιπποκράτειο)', 'Αθήνα (Γκύζη)', 'Αθήνα (Ιπποκράτους)',
       'Αθήνα (Ελαιώνας)', 'Αθήνα (Άγιος Σώστης)', 'Αθήνα (Εξάρχεια)',
       'Αθήνα (Πεδίον Άρεως)', 'Αθήνα (Πλατεία Αττικής)',
       'Αθήνα (Άνω Κυψέλη)', 'Αθήνα (Κεραμεικός)', 'Αθήνα (Άγιος Λουκάς)',
       'Αθήνα (Κέντρο)', 'Αθήνα (Ηπείρου)', 'Αθήνα (Ακαδημία Πλάτωνος)',
       'Αθήνα (Κουκάκι)', 'Αθήνα (Κυψέλη)', 'Αθήνα (Προφήτης Ηλίας)',
       'Αθήνα (Κάτω Πετράλωνα)', 'Αθήνα (Κολωνάκι)', 'Αθήνα (Παγκράτι)',
       'Αθήνα (Άγιος Θωμάς)', 'Αθήνα (Πλατεία Αμερικής)',
       'Αθήνα (Παναθηναϊκό Στάδιο)', 'Αθήνα (Ιλίσια)', 'Αθήνα (Θησείο)',
       'Αθήνα (Χίλτον)', 'Αθήνα (Κολιάτσου)', 'Αθήνα (Περιβόλια)',
       'Αθήνα (Άγιος Ιωάννης)', 'Αθήνα (Λόφος Λαμπράκη)',
       'Αθήνα (Άγιος Ελευθέριος)', 'Αθήνα (Βαρνάβα)',
    

In [40]:
df_useful[['dimos', 'perioxi']] = df_useful['address'].str.extract(r'(\w+)(?:\s+\(([\w\s]+)\))?')

In [41]:
df_useful.head()

,id,internal_id,title_abbreviation,address,price,price_per_unit_area,size_with_square_meter,construction_year,levels,bedrooms,bathrooms,date,extraction_date,geo_lat,geo_lng,company_title,account_id,ad_group_id,dimos,perioxi
0,815053331,47143151,Διαμέρισμα 54 τ.μ.,Αθήνα (Λυκαβηττός),560 €,10,54 τ.μ.,1970.0,"['Υπόγειο', 'Ισόγειο']",1.0,1.0,πριν από 2 ημέρες,2023-06-01,37.979777,23.742884,NaN,2267566.0,NaN,Αθήνα,Λυκαβηττός
1,815047933,47142491,Διαμέρισμα 45 τ.μ.,Αθήνα (Άνω Πατήσια),420 €,9,45 τ.μ.,1969.0,['1ος'],1.0,1.0,πριν από 2 ημέρες,2023-06-01,38.017481,23.735684,NaN,1407351.0,459093.0,Αθήνα,Άνω Πατήσια
2,815017929,47140206,Διαμέρισμα 60 τ.μ.,Αθήνα (Κουντουριώτικα),780 €,13,60 τ.μ.,1975.0,['2ος'],1.0,1.0,πριν από 2 ημέρες,2023-06-01,37.992291,23.755579,NaN,110841.0,NaN,Αθήνα,Κουντουριώτικα
3,814952950,47138240,Διαμέρισμα 55 τ.μ.,Αθήνα (Πολύγωνο),600 €,11,55 τ.μ.,1981.0,['1ος'],1.0,1.0,πριν από 3 ημέρες,2023-06-01,37.999083,23.755417,NaN,1344554.0,NaN,Αθήνα,Πολύγωνο
4,814956607,47138448,Διαμέρισμα 50 τ.μ.,Αθήνα (Πανόρμου),790 €,16,50 τ.μ.,1970.0,['1ος'],1.0,1.0,πριν από 3 ημέρες,2023-06-01,37.990937,23.765072,BLUEGROUND REAL ESTATE,1220409.0,NaN,Αθήνα,Πανόρμου
